# Various methods (you must run all cells)

## Define the styles for all plots

In [130]:
method_styles = {
    'Saliency': {'color':'#800000','marker':'s'},
    'IxG-raw':      {'color':'#ff7f00','marker':'o'},
    'IxG-abs':      {'color':'#ffff99','marker':'s'},
    'IxG-SG-raw':   {'color':'#fb9a99','marker':'o'},
    'IxG-SG-abs':   {'color':'#999999','marker':'s'},
    'IG-raw':       {'color':'#b15928','marker':'o'},
    'IG-abs':       {'color':'#2c7bb6','marker':'s'},
    'IG-U-raw':     {'color':'#6a3d9a','marker':'o'},
    'IG-U-abs':     {'color':'#e31a1c','marker':'s'},
    'IG-SG-raw':    {'color':'#1f78b4','marker':'o'},
    'IG-SG-abs':    {'color':'#fdbf6f','marker':'s'},
    'IG-SG-SQ':     {'color':'#08519c','marker':'s'},
    'Grad-CAM':     {'color':'#ff7f00','marker':'D'},
    'Grad-CAMpp':   {'color':'#cab2d6','marker':'D'},
    'SG-CAMpp':     {'color':'#b2df8a','marker':'D'},
    'XG-CAM':       {'color':'#fb9a99','marker':'D'},
    'Layer-CAM':    {'color':'#8c510a','marker':'D'},
    'Edge-Sobel':          {'color':'#1a9850','marker':'>'},
    'Edge-Gradient':       {'color':'#d73027','marker':'>'},
    'Edge-Canny':          {'color':'#4575b4','marker':'>'},
    'Edge-Marr-Hildreth':  {'color':'#fdae61','marker':'>'},
    'Frequency-low':   {'color':'#17becf','marker':'+'},
    'Frequency-band':  {'color':'#bcbd22','marker':'+'},
    'Frequency-high':  {'color':'#e7ba52','marker':'+'},
}

grad_family = {'Saliency', 'IxG-raw', 'IxG-abs', 'IxG-SG-raw', 'IxG-SG-abs'}
ig_family   = [m for m in method_styles if m.startswith('IG-')]
cam_family  = [m for m in method_styles if 'CAM' in m]

# enforce markers
for m in grad_family:
    # circle for all gradient-based
    method_styles[m]['marker'] = 'o'

for m in ig_family:
    # square for all IG-based
    method_styles[m]['marker'] = 's'

for m in cam_family:
    # diamond for all CAM-based
    method_styles[m]['marker'] = 'D'

## Utils

In [131]:
import pandas as pd
import matplotlib.pyplot as plt

# Returns data dictionary: method→{model→value} dict

def load_data(file_path):
    df = pd.read_csv(file_path, header=None, names=['model','method','value'])
    df['method'] = df['method'].replace({'Gradient-abs': 'Saliency'})
    df['method'] = df['method'].replace({'IG-SG-SQ-abs': 'IG-SG-SQ'})
    df['method'] = df['method'].str.replace(r'^Dummy-', '', regex=True)

    method_dict = {}
    for _,row in df.iterrows():
        method_dict.setdefault(row.method, {})[row.model] = row.value

    all_methods = set(df['method'].unique())

    x_values = df['value']
    
    return method_dict, all_methods, x_values

In [132]:
## Hardcoded models so they are shown in the desired order and mapped to nice names

def get_y_map():
    model_order = [
        'resnet18', 'resnet50', 'resnet101', 'resnet152',
        'wide_resnet50_2', 'fixup_resnet50', 'x_resnet50',
        'vgg11', 'vgg13', 'vgg16', 'vgg19',
        'vgg16_bn', 'x_vgg16',
    ]
    model_order.reverse() 

    label_map = {
        'resnet18':        'ResNet-18',
        'resnet50':        'ResNet-50',
        'resnet101':       'ResNet-101',
        'resnet152':       'ResNet-152',
        'wide_resnet50_2': 'Wide ResNet-50',
        'fixup_resnet50':  'Fixup ResNet-50',
        'x_resnet50':      'xResNet-50',
        'vgg11':           'VGG-11',
        'vgg13':           'VGG-13',
        'vgg16':           'VGG-16',
        'vgg19':           'VGG-19',
        'vgg16_bn':        'VGG-16 (BN)',
        'x_vgg16':         'xVGG-16',
    }

    return {
        model: (i, label_map[model])
        for i, model in enumerate(model_order)
    }

In [133]:
def get_y_map_cifar():
    model_order = [
        'cifar100_resnet20', 'cifar100_resnet32',
        'cifar100_resnet44', 'cifar100_resnet56',
        'cifar100_vgg11_bn','cifar100_vgg13_bn',
        'cifar100_vgg16_bn','cifar100_vgg19_bn',
    ]
    # if you like ascending on the y‐axis:
    model_order.reverse()
    
    name_map = {
        'cifar100_resnet20':  'ResNet-20',
        'cifar100_resnet32':  'ResNet-32',
        'cifar100_resnet44':  'ResNet-44',
        'cifar100_resnet56':  'ResNet-56',
        'cifar100_vgg11_bn':  'VGG-11 (BN)',
        'cifar100_vgg13_bn':  'VGG-13 (BN)',
        'cifar100_vgg16_bn':  'VGG-16 (BN)',
        'cifar100_vgg19_bn':  'VGG-19 (BN)',
    }

    return {
        model: (i, name_map[model])
        for i, model in enumerate(model_order)
    }

In [134]:
def should_skip(method):
    if method in keep_methods:
        return False
    for pref in keep_prefixes:
        if method.startswith(pref):
            return False
    if method in skip_methods:
            return True
    return any(method.startswith(p) for p in skip_prefixes)

In [135]:
def plot(y_map, all_methods, method_styles, x_values, filename):
    
    fig, ax = plt.subplots(figsize=(8,6))
    for model, (y, label) in y_map.items():
        ax.axhline(y, color='lightgray', linestyle='--', zorder=0)

    items = sorted(y_map.items(), key=lambda kv: kv[1])
    prev = {}
    for model, (y, label) in items:
        
        for method, scores in method_dict.items():
            
            if should_skip(method):
              continue
    
            if model not in scores: 
              continue
                
            x = scores[model]
    
            # --- override for Dummy families ---
            if method.startswith('Entropy'):
                style = {'color':'black','marker':'X'}  # big X
            elif method.startswith('Gaussian'):
                style = {'color':'black','marker':'v'}  # downward triangle
            elif method.startswith('Random'):
                style = {'color':'black','marker':'*'}  # star
            else:
                style = method_styles.get(method, {'color':'black','marker':'o'})

            ax.plot(x, y, marker=style['marker'],
                    color=style['color'], markersize=8, linestyle='', zorder=3)

            if method in prev:
                x0, y0 = prev[method]
                ax.plot([x0, x], [y0, y],
                        color=style['color'],
                        linewidth=1,
                        alpha=0.6,
                        zorder=2)

            prev[method] = (x, y)

    ax.set_yticks([y for _, (y, _) in items])
    ax.set_yticklabels([label for _, (_, label) in items])
    ax.set_xlabel('Attribution score')
    ax.set_title('IDSDS - Ascending')
    rng = x_values
    ax.set_xlim(rng.min()-0.05, rng.max()+0.05)
    plt.tight_layout()

    fig.savefig(f"visualisations/{filename}", dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    plt.show()
    

# Show Plots

## Load the data (make sure you run it before you run the plots)

In [136]:
file_path = 'results/1 evaluate_all_methods_with_baseline - results.csv' 
method_dict, all_methods, x_values = load_data(file_path)
y_map = get_y_map()

In [137]:
# We always need to skip Gradient-raw
skip_methods  = {'Gradient-raw'}
skip_prefixes = []
keep_methods = set()
keep_prefixes = []

## Show all plots

In [138]:
plot(y_map, all_methods, method_styles, x_values, "evaluate_all_methods_with_baseline/all-evaluations.png")

## Show only dummy plots

In [139]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'Frequency-low','Frequency-band','Frequency-high', 'Edge-Sobel', 'Edge-Gradient', 'Edge-Canny', 'Edge-Marr-Hildreth'}
keep_prefixes = ['Entropy','Gaussian','Random']
skip_methods -= keep_methods

plot(y_map, all_methods, method_styles, x_values, "evaluate_all_methods_with_baseline/dummy_only.png")

## Show only Saliency and IxG

In [140]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'Saliency','IxG-raw','IxG-SG-raw','IxG-abs', 'IxG-SG-abs'}
keep_prefixes = []
skip_methods -= keep_methods

plot(y_map, all_methods, method_styles, x_values, "evaluate_all_methods_with_baseline/saliency_IxG.png")

##  Show only IG

In [141]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'IG-raw','IG-abs','IG-U-raw','IG-U-abs', 'IG-SG-raw', 'IG-SG-abs', 'IG-SG-SQ'}
keep_prefixes = []
skip_methods -= keep_methods

plot(y_map, all_methods, method_styles, x_values, "evaluate_all_methods_with_baseline/IG_only.png")

## Show only CAM

In [142]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'Grad-CAM','Grad-CAMpp','SG-CAMpp','XG-CAM', 'Layer-CAM'}
keep_prefixes = []

plot(y_map, all_methods, method_styles, x_values, "CAM_only.png")

# Show Cifar plot

In [143]:
file_path = 'results/3 train_and_evaluate_cifar - results.csv' 
method_dict, all_methods, x_values = load_data(file_path)
y_map = get_y_map_cifar()

In [144]:
skip_methods  = {'Gradient-raw'}
skip_prefixes = []
keep_methods = set()
keep_prefixes = []
plot(y_map, all_methods, method_styles, x_values, "cifar-all-evaluations.png")

## Saliency only

In [145]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'Saliency','IxG-raw','IxG-SG-raw','IxG-abs', 'IxG-SG-abs'}
keep_prefixes = []
skip_methods -= keep_methods

plot(y_map, all_methods, method_styles, x_values, "cifar_saliency_IxG.png")

## IG only

In [146]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'IG-raw','IG-abs','IG-U-raw','IG-U-abs', 'IG-SG-raw', 'IG-SG-abs', 'IG-SG-SQ'}
keep_prefixes = []
skip_methods -= keep_methods

plot(y_map, all_methods, method_styles, x_values, "cifar_IG_only.png")

## Only CAM

In [147]:
skip_methods = set(all_methods)
skip_prefixes = []
keep_methods = {'Grad-CAM','Grad-CAMpp','SG-CAMpp','XG-CAM', 'Layer-CAM'}
keep_prefixes = []

plot(y_map, all_methods, method_styles, x_values, "cifar_CAM_only.png")